<a href="https://colab.research.google.com/github/thanhnguyen0612/CS2225.CH1507/blob/master/%5BCS2225%5DTesting_6_fruits_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Cài đặt Object detection API:

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

#2. Download model đã được nhóm train sẵn và upload lên goodle drive:


In [ ]:
import gdown

!mkdir '/content/trained_model'; # tạo folder 'trained_model' trên google instance, để chứa model download

url = 'https://drive.google.com/uc?id=1WCOahi-cR1p88MtyH9QdgW465Quqenz4' #URL cố định dùng để download.
output = '/content/trained_model/trained_model.zip' 
gdown.download(url, output, quiet=False)


In [ ]:
# Unzip và remove zip file. Kết quả sẽ được folder CS2225 trên google instance, trong folder này chứ model đã được train sẵn của nhóm.
!unzip '/content/trained_model/trained_model.zip'
!rm -r '/content/trained_model'

#3. Import thư viện và config cần thiết trước khi run test:



In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
#Hàm load image
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
pipeline_file = '/content/CS225/result_model/pipeline.config'
#recover our saved model
pipeline_config = pipeline_file

#
model_dir = '/content/CS225/output_model/ckpt-3'
configs = config_util.get_configs_from_pipeline_file(pipeline_file)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/content/CS225/output_model/ckpt-3'))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap('/content/CS225/fruit_label_map.pbtxt')
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

#4. Cách test 1: test bằng cách input 01 hình ảnh:
* Note: có 2 cách test: input hình ảnh và chụp hình từ webcam (xem mục số 5)
* Trong folder 'CS225/test_images/' **(folder này đã được download ở mục 2)** nhóm đã chuẩn bị sẵn 1 số hình ngẫu nhiên để thuận tiện việc test.
* Cách run test: Gán biến **img** bằng đường dẫn dẫn tới hình ảnh muốn test **(hình ảnh nằm trên google instance đang connect)**

In [ ]:
import random
import glob
import cv2

#Randomly chọn hình ảnh trong folder CS2225/test_images
TEST_IMAGE_PATHS = glob.glob('/content/CS2225/test_images/*.jpg')
img = random.choice(TEST_IMAGE_PATHS)

image_np = load_image_into_numpy_array(img)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
      line_thickness=10
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

#5. Cách test 2: Lấy hình được chụp từ webcam
* Webcam sẽ được bật, click 1 click để chụp hình từ webcam
* Output: hình ảnh được chụp cùng với bounding box, label name, score

In [ ]:
# Define hàm take_photo: làm nhiệm vụ bật webcam & chụp hình
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
from PIL import Image

VIDEO_HTML = """
<div class="video_container">
  <video autoplay
   width=%d height=%d></video>
  <div style='position: absolute;top: 40px; left: 40px; font-size: 40px; color: green;'>Click to save!</div>
</div>
<script>
var video = document.querySelector('video')
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)
  
var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""
OUTPUT_IMG_FROM_WEBCAM_PATH = '/content/test_img.jpg'
def take_photo(filename=OUTPUT_IMG_FROM_WEBCAM_PATH, quality=0.8, size=(800,600)):
  handle = display(HTML(VIDEO_HTML % (size[0],size[1],quality)), display_id='videoHTML')
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  if filename:
    f = io.BytesIO(binary)
    Image.open(f).save(filename)
  else:
    f = io.BytesIO(binary)
    return np.asarray(Image.open(f))

In [ ]:
import random
import glob
import cv2

take_photo()
img = OUTPUT_IMG_FROM_WEBCAM_PATH
image_np = load_image_into_numpy_array(img)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
      line_thickness=12
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()